In [422]:
# GENERAL USE LIBRARY IMPORTS

import datetime as dt
from datetime import timedelta
import random
import random as rand
import warnings
from datetime import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import pandas as pd
import os
import requests
import re
from zipfile import ZipFile

warnings.filterwarnings('ignore')

In [74]:
# Create dictionary of team abbvs and names
team_abbvs = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK',  'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC','SAS', 'TOR', 'UTA','WAS']
team_names = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets','Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']
name_to_abbv = dict(zip(team_names,team_abbvs))

In [489]:

# create list of dates:
# Define the start and end dates
start_date = '2023-10-24'
#testing with smaller set
#end_date = '2023-12-25'
end_date = '2024-03-31'

# Generate the range of dates
date_range = pd.date_range(start=start_date, end=end_date)

# Convert to list
date_list = list(date_range)

# format dates:
formatted_date_list = [date.strftime('%Y%m%d') for date in date_range]

# Print the formatted list of dates
print(formatted_date_list)
# Print the list of dates


['20231024', '20231025', '20231026', '20231027', '20231028', '20231029', '20231030', '20231031', '20231101', '20231102', '20231103', '20231104', '20231105', '20231106', '20231107', '20231108', '20231109', '20231110', '20231111', '20231112', '20231113', '20231114', '20231115', '20231116', '20231117', '20231118', '20231119', '20231120', '20231121', '20231122', '20231123', '20231124', '20231125', '20231126', '20231127', '20231128', '20231129', '20231130', '20231201', '20231202', '20231203', '20231204', '20231205', '20231206', '20231207', '20231208', '20231209', '20231210', '20231211', '20231212', '20231213', '20231214', '20231215', '20231216', '20231217', '20231218', '20231219', '20231220', '20231221', '20231222', '20231223', '20231224', '20231225', '20231226', '20231227', '20231228', '20231229', '20231230', '20231231', '20240101', '20240102', '20240103', '20240104', '20240105', '20240106', '20240107', '20240108', '20240109', '20240110', '20240111', '20240112', '20240113', '20240114', '20

## Team stats

In [486]:
## update to change from zips
## for file names
def update_team_from_boxscore(data,date, away, home ): # path is  NBA_Box_Scores/date/away@home
    #data = pd.DataFrame()
    base_directory = "NBA_Box_Scores"
    directory_path = f"{base_directory}/{date}/{away}@{home}"
    boxscore_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    for file_name in boxscore_files:
        if file_name == "away_team.csv":
            team_name = away
        elif file_name == "home_team.csv":
            team_name = home
        else:
            print("error no csv found")
            continue
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        team_stats_row = df.iloc[-1]
        # convert objects into floats
        #numeric_columns = team_stats_row.drop(['Players'])  # Assuming 'Team' and 'Games Played' are not to be converted
        #team_stats_row[numeric_columns.index] = numeric_columns.astype(float)
        
        if team_name in data.index:
            data = update_running_team_averages(data, team_name, team_stats_row)
        else:
            # Add new team
            data = add_new_team(data,team_name, team_stats_row)

    return data
    
        

def update_running_team_averages(data, team_name, team_stats_row):
    """Update team stats and calculate running averages."""
    existing_stats = data.loc[team_name]
    games_played = existing_stats['Games Played'] + 1
    for col in team_stats_row.index:
        new_value = pd.to_numeric(team_stats_row[col], errors='coerce')
        old_avg = pd.to_numeric(existing_stats[col], errors='coerce')
        if pd.notna(new_value) and pd.notna(old_avg):
            new_avg = ((old_avg * (games_played - 1)) + new_value) / games_played
            data.at[team_name, col] = new_avg
     ## fixing for percentages
        if re.search(r'%',col):
            if col == 'FG%':
                actual = data.at[team_name,'FG']/data.at[team_name,'FGA']
                data.at[team_name,col]= actual
            if col == '3P%':
                actual = data.at[team_name,'3P']/data.at[team_name,'3PA']
                data.at[team_name,col]= actual
            if col == 'FT%':
                actual = data.at[team_name,'FT']/data.at[team_name,'FTA']
                data.at[team_name,col]= actual
            if col == 'TS%':
                actual = data.at[team_name,'PTS']/(2*(data.at[team_name,'FGA']+ 0.44 *data.at[team_name,'FTA']))
                data.at[team_name,col]= actual
            if col == 'eFG%':
                actual = (data.at[team_name,'FG']+ 0.5 * data.at[team_name,'3P'])/data.at[team_name,'FGA']
                data.at[team_name,col]= actual
    data.at[team_name, 'Games Played'] = games_played
    return data

def add_new_team(data, team_name, team_stats_row):
    """Add a new team to the DataFrame."""
    team_stats_row['Games Played'] = 1  # Initialize games played
    team_name = [team_name]
    team_stats_df = pd.DataFrame([team_stats_row], index = team_name)
    data = pd.concat([data, team_stats_df])
    return data

In [491]:
## creates cumul team stats for each date
all_team_stats = pd.DataFrame()
main_folder = "NBA_Team_Statsheet"
os.makedirs(main_folder, exist_ok = True)
for date in formatted_date_list:
    date_folder = f"{date}"
    full_folder_path = os.path.join(main_folder, date_folder)
    os.makedirs(full_folder_path, exist_ok = True)
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        away_abbr = row['Away']
        

        all_team_stats = update_team_from_boxscore(all_team_stats,date,away_abbr,home_abbr)
    all_team_stats.to_csv(f"{full_folder_path}/statsheet.csv")
        
#print(all_team_stats)       

In [390]:
# get rid of null and unnessary values
def clean_team_stats(data):
    new_df = data.drop(['Players','MP','AST','+/-','MP.1','AST%','BPM'],axis = 1, inplace = True)
    return new
new_df = clean_team_stats(all_team_stats)
all_team_stats

,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,...,ORB%,DRB%,TRB%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,Games Played
DEN,47.25,87.75,0.53875,12.0,31.5,0.3795,9.75,15.0,0.66875,8.0,...,21.375,74.075,50.575,8.125,12.675,12.3,100.0,119.375,104.6,4
LAL,40.5,87.75,0.464,9.5,32.5,0.2865,19.5,25.25,0.7715,8.0,...,17.55,79.625,49.5,7.425,11.2,12.325,100.0,107.3,109.5,4
GSW,42.5,91.75,0.465,14.25,40.25,0.35925,16.25,21.25,0.769,13.5,...,28.4,74.15,52.425,9.15,9.7,12.65,100.0,115.925,104.975,4
PHO,40.333333,87.0,0.463667,10.0,33.666667,0.297667,19.0,22.666667,0.825667,11.333333,...,25.666667,73.866667,51.733333,7.333333,10.2,14.6,100.0,109.6,102.5,3
ORL,38.666667,87.0,0.446333,10.666667,32.333333,0.328667,19.0,25.333333,0.754333,13.0,...,29.933333,82.166667,55.366667,9.7,9.333333,11.766667,100.0,111.833333,100.466667,3
HOU,39.333333,89.666667,0.434333,9.666667,33.0,0.292,12.666667,18.333333,0.690667,9.333333,...,20.233333,70.4,43.8,6.733333,6.866667,12.7,100.0,101.1,116.866667,3
NYK,37.333333,92.333333,0.405333,15.0,40.666667,0.361,16.0,24.0,0.676667,15.0,...,28.333333,83.733333,52.966667,8.4,5.5,12.233333,100.0,106.333333,108.833333,3
BOS,44.333333,91.333333,0.485,15.666667,43.666667,0.358667,13.333333,17.333333,0.748,12.666667,...,28.333333,74.733333,53.5,8.066667,14.033333,13.133333,100.0,119.333333,108.966667,3
IND,46.666667,97.666667,0.474,15.666667,42.333333,0.373667,15.333333,18.0,0.860333,10.666667,...,24.033333,74.8,51.066667,6.333333,8.033333,10.2,100.0,120.7,111.9,3
WAS,41.0,92.0,0.445333,12.0,35.333333,0.343,19.333333,25.666667,0.752667,11.0,...,22.2,76.066667,46.966667,8.7,11.233333,11.433333,100.0,108.4,119.266667,3


## player stats

In [483]:
## update to change from zips
## for file names
def update_player_from_boxscore(data, date, away, home ): # path is  NBA_Box_Scores/date/away@home
    #data = pd.DataFrame()
    base_directory = "NBA_Box_Scores"
    directory_path = f"{base_directory}/{date}/{away}@{home}"
    boxscore_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    for file_name in boxscore_files:
    
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        player_rows = df.iloc[:-1]

        #player_names = player_rows['Players'].tolist()
        for i, player_stats_row in player_rows.iterrows():
            player_name = player_stats_row['Players']
            if player_name in data.index:
                data = update_running_player_averages(data, player_name, player_stats_row)
            else:
            # Add new team
                data = add_new_player(data,player_name, player_stats_row)

    return data
    
        

def update_running_player_averages(data, player_name, player_stats_row):
    """Update team stats and calculate running averages."""
    existing_stats = data.loc[player_name]
    mp = player_stats_row['MP']
    if re.search(r'\d', mp):
        games_played = existing_stats['Games Played'] + 1
        for col in player_stats_row.index:
            new_value = pd.to_numeric(player_stats_row[col], errors='coerce')
            old_avg = pd.to_numeric(existing_stats[col], errors='coerce')
            if pd.notna(new_value) and pd.notna(old_avg):
                new_avg = ((old_avg * (games_played - 1)) + new_value) / games_played
                data.at[player_name, col] = new_avg
            ## fixing for percentages
            if re.search(r'%',col):
                if col == 'FG%':
                    actual = data.at[player_name,'FG']/data.at[player_name,'FGA']
                    data.at[player_name,col]= actual
                if col == '3P%':
                    actual = data.at[player_name,'3P']/data.at[player_name,'3PA']
                    data.at[player_name,col]= actual
                if col == 'FT%':
                    actual = data.at[player_name,'FT']/data.at[player_name,'FTA']
                    data.at[player_name,col]= actual
                if col == 'TS%':
                    actual = data.at[player_name,'PTS']/(2*(data.at[player_name,'FGA']+ 0.44 *data.at[player_name,'FTA']))
                    data.at[player_name,col]= actual
                if col == 'eFG%':
                    actual = (data.at[player_name,'FG']+ 0.5 * data.at[player_name,'3P'])/data.at[player_name,'FGA']
                    data.at[player_name,col]= actual    
        data.at[player_name, 'Games Played'] = games_played
    return data

def add_new_player(data, player_name, player_stats_row):
    """Add a new team to the DataFrame."""
    mp = player_stats_row['MP']
    if re.search(r'\d', mp):
        player_stats_row['Games Played'] = 1 # Initialize games played
        player_name = [player_name]
        player_stats_df = pd.DataFrame([player_stats_row], index = player_name)
        data = pd.concat([data, player_stats_df])
    return data



In [492]:
## creates cumul team stats for a date
all_player_stats = pd.DataFrame()
main_folder = "NBA_Player_Statsheet"
for date in formatted_date_list:
    date_folder = f"{date}"
    full_folder_path = os.path.join(main_folder,date_folder)
    os.makedirs(full_folder_path, exist_ok = True)
    games_df = simple_games_df[simple_games_df['Date'] == date]
    for index, row in games_df.iterrows():
        # Format the URL with the current game's home team abbreviation
        home_abbr = row['Home']
        away_abbr = row['Away']

        all_player_stats = update_player_from_boxscore(all_player_stats,date,away_abbr,home_abbr)
    all_player_stats.to_csv(f"{full_folder_path}/statsheet.csv")
#print(all_player_stats)       

In [488]:
all_team_stats.loc['DEN']

Players         Team Totals
MP                    240.0
FG                  44.0625
FGA                 90.0625
FG%                0.489244
3P                 11.78125
3PA                31.28125
3P%                0.376623
FT                 15.21875
FTA                20.65625
FT%                0.736762
ORB                11.46875
DRB                33.15625
TRB                  44.625
AST                 29.3125
STL                 7.15625
BLK                   5.375
TOV                11.40625
PF                  18.9375
PTS                 115.125
+/-                     NaN
MP.1                  240.0
TS%                0.580552
eFG%                0.55465
3PAr               0.347625
FTr                0.233281
ORB%                 26.325
DRB%              75.284375
TRB%                51.0375
AST%              66.240625
STL%               7.384375
BLK%               9.528125
TOV%                10.2875
USG%                  100.0
ORtg                119.375
DRtg              11

In [464]:
file_path = "NBA_Box_Scores/20231024/PHO@GSW/away_team.csv"
df = pd.read_csv(file_path)
new = pd.DataFrame()

player_rows = df.iloc[:-1]
for i, player_stats_row in player_rows.iterrows():
    player_name = player_stats_row['Players']
    for col in player_stats_row.index:
        new_value = pd.to_numeric(player_stats_row[col], errors='coerce')
        new.at[player_name, col] = new_value
        if re.search(r'%',col):
            if col == 'FG%':
                actual = player_stats_row['FG']/player_stats_row['FGA']
                print(actual)

TypeError: unsupported operand type(s) for /: 'str' and 'str'